In [1]:
# 读取图像信息
import shutil
import os


training_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/training/'
train_data_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/train_data/'
dirs = os.listdir(training_path)
for item in dirs:
    item_label = item.split('_')[0]
    des_path = train_data_path+item_label
    if not os.path.exists(des_path):
        os.makedirs(des_path)
    shutil.copyfile(os.path.join(training_path, item), os.path.join(des_path, item))


testing_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/testing/'
test_data_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/test_data/'
dirs = os.listdir(testing_path)
for item in dirs:
    item_label = item.split('_')[0]
    des_path = test_data_path+item_label
    if not os.path.exists(des_path):
        os.makedirs(des_path)
    shutil.copyfile(os.path.join(testing_path, item), os.path.join(des_path, item))

In [2]:
# 

from keras.layers import add, Flatten
from keras.utils import plot_model
from keras.metrics import top_k_categorical_accuracy
from keras.models import load_model

import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Input, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, concatenate, \
    Activation, ZeroPadding2D
import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.optimizers import Adam
# 解决运行时服务挂掉的问题
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [15]:
train_data_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/train_data/'
test_data_path = '/Users/anlei/Documents/AllCode/Jupyter/5DEEP/5Deep-final/images5DEEP/raw_image/test_data/'

In [16]:
# 声明一个ImageDataGenerator类对象，并给出你需要进行的数据增强选项
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
# 调用.flow_from_director()方法，第一个为数据集路径。生成数据集及标签
train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
# 用同样的方法加载测试集
test_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 50000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.


In [3]:
# classes = 10
# batch_size = 64
# epochs = 20

classes = 10
batch_size = 2
epochs = 2

# def resnet_block(inputs, 
#                  num_filters=16, 
#                  kernel_size=3,
#                  strides=1, 
#                  activation='relu'):
#     x = Conv2D(num_filters, 
#                kernel_size=kernel_size, 
#                strides=strides, 
#                padding='same',
#                kernel_initializer='he_normal', 
#                kernel_regularizer=l2(1e-4))(inputs)
#     x = BatchNormalization()(x)
#     if (activation):
#         x = Activation('relu')(x)
#     return x

In [11]:
def Conv2d_BN(x, nb_filter, kernel_size, strides=(1, 1), padding='same', name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None

    x = Conv2D(nb_filter, kernel_size, padding=padding, strides=strides, activation='relu', name=conv_name)(x)
    x = BatchNormalization(axis=3, name=bn_name)(x)
    return x


def identity_Block(inpt, nb_filter, kernel_size, strides=(1, 1), with_conv_shortcut=False):
    x = Conv2d_BN(inpt, nb_filter=nb_filter, kernel_size=kernel_size, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=nb_filter, kernel_size=kernel_size, padding='same')
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt, nb_filter=nb_filter, strides=strides, kernel_size=kernel_size)
        x = add([x, shortcut])
        return x
    else:
        x = add([x, inpt])
        return x

def bottleneck_Block(inpt,nb_filters,strides=(1,1),with_conv_shortcut=False):
    k1,k2,k3=nb_filters
    x = Conv2d_BN(inpt, nb_filter=k1, kernel_size=1, strides=strides, padding='same')
    x = Conv2d_BN(x, nb_filter=k2, kernel_size=3, padding='same')
    x = Conv2d_BN(x, nb_filter=k3, kernel_size=1, padding='same')
    if with_conv_shortcut:
        shortcut = Conv2d_BN(inpt, nb_filter=k3, strides=strides, kernel_size=1)
        x = add([x, shortcut])
        return x
    else:
        x = add([x, inpt])
        return x

def resnet_50(width,height,channel,classes):
    inpt = Input(shape=(width, height, channel))
    x = ZeroPadding2D((3, 3))(inpt)
    x = Conv2d_BN(x, nb_filter=64, kernel_size=(7, 7), strides=(2, 2), padding='valid')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    #conv2_x
    x = bottleneck_Block(x, nb_filters=[64,64,256],strides=(1,1),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[64,64,256])
    x = bottleneck_Block(x, nb_filters=[64,64,256])

    #conv3_x
    x = bottleneck_Block(x, nb_filters=[128, 128, 512],strides=(2,2),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])
    x = bottleneck_Block(x, nb_filters=[128, 128, 512])

    #conv4_x
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024],strides=(2,2),with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])
    x = bottleneck_Block(x, nb_filters=[256, 256, 1024])

    #conv5_x
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048], strides=(2, 2), with_conv_shortcut=True)
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048])
    x = bottleneck_Block(x, nb_filters=[512, 512, 2048])

    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten()(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=inpt, outputs=x)
    return model




# # 建一个20层的ResNet网络 
# def resnet(input_shape):
#     inputs = Input(shape=input_shape)# Input层，用来当做占位使用
    
#     #第一层
#     x = resnet_block(inputs)
#     print('layer1,xshape:',x.shape)
#     # 第2~7层
#     for i in range(6):
#         a = resnet_block(inputs = x)
#         b = resnet_block(inputs=a,activation=None)
#         x = keras.layers.add([x,b])
#         x = Activation('relu')(x)
#     # out：32*32*16
#     # 第8~13层
#     for i in range(6):
#         if i == 0:
#             a = resnet_block(inputs = x,strides=2,num_filters=32)
#         else:
#             a = resnet_block(inputs = x,num_filters=32)
#         b = resnet_block(inputs=a,activation=None,num_filters=32)
#         if i==0:
#             x = Conv2D(32,kernel_size=3,strides=2,padding='same',
#                        kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
#         x = keras.layers.add([x,b])
#         x = Activation('relu')(x)
#     # out:16*16*32
#     # 第14~19层
#     for i in range(6):
#         if i ==0 :
#             a = resnet_block(inputs = x,strides=2,num_filters=64)
#         else:
#             a = resnet_block(inputs = x,num_filters=64)

#         b = resnet_block(inputs=a,activation=None,num_filters=64)
#         if i == 0:
#             x = Conv2D(64,kernel_size=3,strides=2,padding='same',
#                        kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
#         x = keras.layers.add([x,b])# 相加操作，要求x、b shape完全一致
#         x = Activation('relu')(x)
#     # out:8*8*64
#     # 第20层   
#     x = AveragePooling2D(pool_size=2)(x)
#     # out:4*4*64
#     y = Flatten()(x)
#     # out:1024
#     outputs = Dense(10,activation='softmax',
#                     kernel_initializer='he_normal')(y)
    
#     #初始化模型
#     #之前的操作只是将多个神经网络层进行了相连，通过下面这一句的初始化操作，才算真正完成了一个模型的结构初始化
#     model = Model(inputs=inputs,outputs=outputs)
#     return model

In [13]:
# 模型的总结
model = resnet_50(224, 224, 3, 3)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
____________________________________________________________________________________________

In [14]:
# # 动态变化学习率
# def lr_sch(epoch):
#     #200 total
#     if epoch <50:
#         return 1e-3
#     if 50<=epoch<100:
#         return 1e-4
#     if epoch>=100:
#         return 1e-5

In [23]:
model = resnet_50(224, 224, 3, 3)
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy',top_k_categorical_accuracy])

In [24]:
 model.fit_generator(train_generator,
                     validation_data=test_generator,
                     epochs=epochs,
                     steps_per_epoch=5,
                     verbose=1,)


# score = model.evaluate(test_generator, verbose=0)

# checkpoint = ModelCheckpoint(filepath='5deep-resnet.h5', 
#                              monitor='val_accuracy', 
#                              verbose=1,
#                              save_best_only=True)

# lr_scheduler = LearningRateScheduler(lr_sch)

# lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', 
#                                factor=0.2, 
#                                patience=5, 
#                                mode='max', 
#                                min_lr=1e-3)
# callbacks = [checkpoint, lr_scheduler, lr_reducer]

# history = model.fit_generator(train_generator, 
#                               steps_per_epoch = batch_size, 
#                               epochs = epochs, 
#                               validation_data=test_generator, 
#                               verbose=1, 
#                               callbacks=callbacks)


KeyboardInterrupt: 

In [ ]:
# 打印  loss和acc
print('Test loss', score[0])
print('Test accuracy',score[1])

In [ ]:
# 绘图
import matplotlib.pyplot as plt
%matplotlib inline

accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, label='Training accuracy')
plt.plot(epochs, val_accuracy, label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
    